In [3]:
import pickle # pickle list object
import pandas as pd
import numpy as np

#Libraries for ML
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV

In [5]:
# Loading the saved random forest smote model with pickle

model_pickle_path = 'Logistic_Regression.pkl'
model_pickle_path2 = 'XGBoost.pkl'
model_pickle_path3 = 'Random_Forest.pkl'

Logistic_pkl = open(model_pickle_path, 'rb')
Logistic_model = pickle.load(Logistic_pkl)
print("Loaded XGboost model :: ", Logistic_model)

xgboost_pkl = open(model_pickle_path2, 'rb')
xgboost_model = pickle.load(xgboost_pkl)
print("Loaded XGboost model :: ", xgboost_model)

RF_pkl = open(model_pickle_path3, 'rb')
RF_model = pickle.load(RF_pkl)
print("Loaded XGboost model :: ", RF_model)

Loaded XGboost model ::  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=8,
          penalty='l2', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)
Loaded XGboost model ::  XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=nan, n_estimators=100,
       n_jobs=8, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
Loaded XGboost model ::  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
  

## Importing Data 

In [16]:
df = pd.read_csv('Dataframe_after_first_iteration.csv')
df.drop('Probability', inplace = True)

KeyError: "['Probability'] not found in axis"

In [7]:
X = df.drop('radiant_win', axis=1)
y = df.radiant_win
    
X_train, X_test, y_train, y_test = train_test_split(X, y)
    
scaler = StandardScaler()
scaler.fit(X_train)
    
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

/Users/hiowatah/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/hiowatah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':
/Users/hiowatah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
Logistic_model()

## Randomized Search for Random Forest

In [8]:
# Create regularization hyperparameter space
C = C_param_range = [0.001,0.1, 1, 5]

# Create hyperparameter options
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

hyperparameters = dict(C=C, solver = solver)


In [11]:
# Create grid search using 5-fold cross validation
clf_gs = GridSearchCV(Logistic_model, hyperparameters, cv=5, scoring='f1', n_jobs=8, verbose=10) 
model = clf_gs.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   43.1s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  2.5min
/Users/hiowatah/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:  4.9min
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  7.4min
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  9.5min
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed: 13.5min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed: 17.2min
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed: 22.5min
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed: 27.3min
[Parallel(n_jobs=8)]: Done  96 out of 100 | elapsed: 32.0min remain

In [13]:
# View best hyperparameters
print('Best Solver:', model.best_estimator_.get_params()['solver'])
print('Best C:', model.best_estimator_.get_params()['C'])

y_pred_train = model.best_estimator_.predict(X_train)

y_pred_test = model.best_estimator_.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Training F1 Score:",f1_score(y_train, y_pred_train))
print("Testing F1 Score:",f1_score(y_test, y_pred_test))


Best Solver: saga
Best C: 0.001
Training F1 Score: 0.6745186734091507
Testing F1 Score: 0.6701805300670053


In [15]:
model_params = list(zip(X.columns, model.best_estimator_.coef_[0]))

model_params.sort(key = lambda x: abs(x[1]), reverse=True)
model_params

[('Probability', 0.2644579846283792),
 ('Radiant_Dire_gold_dif', 0.2572758592713633),
 ('Radiant_Dire_lh_dif', 0.12769141843584084),
 ('Radiant_Dire_xp_dif', 0.10667020082043566),
 ('Radiant_Dire_winrate_dif', 0.09843165579829617),
 ('Dire_Champion_Winpercent', -0.07204587827280655),
 ('Radiant_Champion_Winpercent', 0.06990468873538833),
 ('Radiant_ward_observer_dif', 0.050518827670484344),
 ('r1_hero_winrate', 0.03549562615498113),
 ('d5_hero_winrate', -0.03485484123838045),
 ('r5_hero_winrate', 0.03321783974352779),
 ('d1_hero_winrate', -0.03200356520702128),
 ('r2_hero_winrate', 0.03184955055579821),
 ('d4_hero_winrate', -0.03077969245432693),
 ('d2_hero_winrate', -0.03050909167078567),
 ('d3_hero_winrate', -0.030309361263207445),
 ('Radiant_FlyingCourier_dif', 0.02758243029373734),
 ('r4_hero_winrate', 0.0268703230019228),
 ('r3_hero_winrate', 0.025867298439710886),
 ('Dire_Hero2_67', -0.023000446715727714),
 ('Radiant_Hero3_2', -0.022099888492865913),
 ('Radiant_tp_dif', -0.021296